In [116]:
import pandas as pd
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
df=pd.read_excel('test1.xlsx')
df.columns = ['Date', 'Value']
pre_days=1
df['label']=df['Value'].shift(-pre_days)
print(df)

        Date   Value   label
0   20160911  621.65  609.67
1   20160912  609.67  610.92
2   20160913  610.92  608.82
3   20160914  608.82  610.38
4   20160915  610.38  609.11
5   20160916  609.11  607.04
6   20160917  607.04  611.58
7   20160918  611.58  610.19
8   20160919  610.19  608.66
9   20160920  608.66  598.88
10  20160921  598.88  597.42
11  20160922  597.42  594.08
12  20160923  594.08  603.88
13  20160924  603.88  601.74
14  20160925  601.74  598.98
15  20160926  598.98  605.96
16  20160927  605.96  605.67
17  20160928  605.67  603.85
18  20160929  603.85  609.39
19  20160930  609.39  614.82
20  20161001  614.82  612.98
21  20161002  612.98  611.85
22  20161003  611.85  609.62
23  20161004  609.62  607.18
24  20161005  607.18  612.08
25  20161006  612.08  617.21
26  20161007  617.21  614.74
27  20161008  614.74  615.65
28  20161009  615.65     NaN


In [118]:
scaler=StandardScaler()           
sca_X=scaler.fit_transform(df.iloc[:,1:-1])
print(sca_X)

[[ 2.17987399]
 [ 0.20688501]
 [ 0.4127478 ]
 [ 0.06689831]
 [ 0.32381507]
 [ 0.11465848]
 [-0.22625031]
 [ 0.52144335]
 [ 0.29252393]
 [ 0.04054787]
 [-1.5701226 ]
 [-1.81057034]
 [-2.36063571]
 [-0.74667144]
 [-1.09910854]
 [-1.55365358]
 [-0.40411576]
 [-0.45187592]
 [-0.75161215]
 [ 0.16077174]
 [ 1.0550397 ]
 [ 0.75200967]
 [ 0.56590971]
 [ 0.19865049]
 [-0.20319367]
 [ 0.60378846]
 [ 1.44864936]
 [ 1.04186448]
 [ 1.19173259]]


In [119]:
mem_his_days=10
deq=deque(maxlen=mem_his_days)
X = []
for i in sca_X:
    deq.append(i)
    if len(deq)==mem_his_days:
        X.append(list(deq))

X_lately=X[-pre_days:]    
X=X[:-pre_days]
print(len(X_lately))
print(len(X))

y=df['label'].values[mem_his_days-1:-pre_days]
print(len(y))

1
19
19


In [120]:
X=np.array(X)
y=np.array(y)
print(X.shape)

(19, 10, 1)


In [121]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

In [122]:
print(len(X_train))

17


In [123]:
model=Sequential()
model.add(LSTM(12,input_shape=X.shape[1:],activation='relu',return_sequences=True)) 
model.add(Dropout(0.1))

model.add(LSTM(12,activation='relu',return_sequences=True)) 
model.add(Dropout(0.1))

model.add(LSTM(12,activation='relu')) 
model.add(Dropout(0.1))

model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mae',metrics=['mape'])

model.fit(X_train, y_train, batch_size=128, epochs=250,verbose=1,validation_data=(X_test, y_test))

Train on 17 samples, validate on 2 samples
Epoch 1/250
17/17 [==============================] - 1s 72ms/step - loss: 607.6363 - mape: 100.0000 - val_loss: 603.0762 - val_mape: 99.9994
Epoch 2/250
17/17 [==============================] - 0s 1ms/step - loss: 607.6332 - mape: 99.9995 - val_loss: 603.0734 - val_mape: 99.9989
Epoch 3/250
17/17 [==============================] - 0s 1ms/step - loss: 607.6306 - mape: 99.9990 - val_loss: 603.0706 - val_mape: 99.9984
Epoch 4/250
17/17 [==============================] - 0s 1ms/step - loss: 607.6277 - mape: 99.9986 - val_loss: 603.0677 - val_mape: 99.9980
Epoch 5/250
17/17 [==============================] - 0s 995us/step - loss: 607.6263 - mape: 99.9983 - val_loss: 603.0648 - val_mape: 99.9975
Epoch 6/250
17/17 [==============================] - 0s 1ms/step - loss: 607.6250 - mape: 99.9981 - val_loss: 603.0618 - val_mape: 99.9970
Epoch 7/250
17/17 [==============================] - 0s 1ms/step - loss: 607.6190 - mape: 99.9971 - val_loss: 603.0588 

Epoch 59/250
17/17 [==============================] - 0s 645us/step - loss: 605.6907 - mape: 99.6784 - val_loss: 601.4775 - val_mape: 99.7334
Epoch 60/250
17/17 [==============================] - 0s 763us/step - loss: 605.4849 - mape: 99.6450 - val_loss: 601.2249 - val_mape: 99.6913
Epoch 61/250
17/17 [==============================] - 0s 763us/step - loss: 604.8480 - mape: 99.5383 - val_loss: 600.9163 - val_mape: 99.6398
Epoch 62/250
17/17 [==============================] - 0s 704us/step - loss: 604.7188 - mape: 99.5167 - val_loss: 600.5287 - val_mape: 99.5751
Epoch 63/250
17/17 [==============================] - 0s 704us/step - loss: 604.0342 - mape: 99.4047 - val_loss: 600.0413 - val_mape: 99.4937
Epoch 64/250
17/17 [==============================] - 0s 645us/step - loss: 603.8795 - mape: 99.3777 - val_loss: 599.4218 - val_mape: 99.3902
Epoch 65/250
17/17 [==============================] - 0s 763us/step - loss: 601.3222 - mape: 98.9555 - val_loss: 598.6608 - val_mape: 99.2631
Epoch 

Epoch 117/250
17/17 [==============================] - 0s 704us/step - loss: 253.3494 - mape: 41.5501 - val_loss: 188.0347 - val_mape: 30.8090
Epoch 118/250
17/17 [==============================] - 0s 704us/step - loss: 276.4078 - mape: 45.4959 - val_loss: 191.1025 - val_mape: 31.3660
Epoch 119/250
17/17 [==============================] - 0s 763us/step - loss: 338.1305 - mape: 55.7558 - val_loss: 191.8851 - val_mape: 31.5338
Epoch 120/250
17/17 [==============================] - 0s 763us/step - loss: 276.7102 - mape: 45.4958 - val_loss: 193.7567 - val_mape: 31.8927
Epoch 121/250
17/17 [==============================] - 0s 704us/step - loss: 288.1837 - mape: 47.5321 - val_loss: 191.8125 - val_mape: 31.5979
Epoch 122/250
17/17 [==============================] - 0s 646us/step - loss: 251.2517 - mape: 41.3809 - val_loss: 185.4830 - val_mape: 30.5461
Epoch 123/250
17/17 [==============================] - 0s 705us/step - loss: 208.7705 - mape: 34.3513 - val_loss: 180.1434 - val_mape: 29.6643

Epoch 175/250
17/17 [==============================] - 0s 704us/step - loss: 78.2867 - mape: 12.8741 - val_loss: 94.0967 - val_mape: 15.5574
Epoch 176/250
17/17 [==============================] - 0s 763us/step - loss: 170.1228 - mape: 27.9991 - val_loss: 76.6997 - val_mape: 12.6766
Epoch 177/250
17/17 [==============================] - 0s 704us/step - loss: 145.4908 - mape: 23.9807 - val_loss: 71.6945 - val_mape: 11.8498
Epoch 178/250
17/17 [==============================] - 0s 645us/step - loss: 117.7942 - mape: 19.3888 - val_loss: 69.2241 - val_mape: 11.4435
Epoch 179/250
17/17 [==============================] - 0s 586us/step - loss: 62.7607 - mape: 10.3488 - val_loss: 74.0577 - val_mape: 12.2490
Epoch 180/250
17/17 [==============================] - 0s 645us/step - loss: 94.3376 - mape: 15.4715 - val_loss: 93.0953 - val_mape: 15.4065
Epoch 181/250
17/17 [==============================] - 0s 704us/step - loss: 81.4242 - mape: 13.4531 - val_loss: 114.0367 - val_mape: 18.8802
Epoch 182

Epoch 233/250
17/17 [==============================] - 0s 704us/step - loss: 141.3126 - mape: 23.1626 - val_loss: 80.0337 - val_mape: 13.2521
Epoch 234/250
17/17 [==============================] - 0s 763us/step - loss: 152.3380 - mape: 25.0881 - val_loss: 70.4594 - val_mape: 11.6637
Epoch 235/250
17/17 [==============================] - 0s 645us/step - loss: 37.6988 - mape: 6.2211 - val_loss: 72.5565 - val_mape: 12.0109
Epoch 236/250
17/17 [==============================] - 0s 645us/step - loss: 60.0587 - mape: 9.9018 - val_loss: 85.2414 - val_mape: 14.1146
Epoch 237/250
17/17 [==============================] - 0s 764us/step - loss: 83.3124 - mape: 13.7041 - val_loss: 96.0699 - val_mape: 15.9105
Epoch 238/250
17/17 [==============================] - 0s 704us/step - loss: 118.7834 - mape: 19.5198 - val_loss: 98.8277 - val_mape: 16.3676
Epoch 239/250
17/17 [==============================] - 0s 765us/step - loss: 92.9570 - mape: 15.2968 - val_loss: 99.4782 - val_mape: 16.4753
Epoch 240/25